In [34]:
import os
import email
import imaplib
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import parseaddr
from email import encoders
import subprocess
import time
import msvcrt

class GmailHandler:
    def __init__(self):
        self.EMAIL = "shobika0531@gmail.com"  
        self.PASSWORD = "teyj gymt xwzo dwim"
        self.IMAP_SERVER = "imap.gmail.com"
        self.SMTP_SERVER = "smtp.gmail.com"
        self.SMTP_PORT = 587

    def connect_imap(self):
        """Establish connection to Gmail's IMAP server"""
        try:
            imap = imaplib.IMAP4_SSL(self.IMAP_SERVER)
            imap.login(self.EMAIL, self.PASSWORD)
            return imap
        except Exception as e:
            print(f"IMAP Connection Error: {str(e)}")
            return None

    def connect_smtp(self):
        """Establish connection to Gmail's SMTP server"""
        try:
            smtp = smtplib.SMTP(self.SMTP_SERVER, self.SMTP_PORT)
            smtp.starttls()
            smtp.login(self.EMAIL, self.PASSWORD)
            return smtp
        except Exception as e:
            print(f"SMTP Connection Error: {str(e)}")
            return None

    def get_attachments(self, email_message):
        """Extract attachments from email message"""
        attachments = []

        for part in email_message.walk():
            if part.get_content_maintype() == 'multipart':
                continue
            if part.get('Content-Disposition') is None:
                continue

            filename = part.get_filename()
            if filename:
                print(f"Found attachment: {filename}")
                attachments.append({
                    'filename': filename,
                    'content': part.get_payload(decode=True),
                    'content_type': part.get_content_type()
                })

        return attachments

    def save_attachment(self, attachment, download_folder="attachments"):
        """Save the attachment to a local folder"""
        if not os.path.isdir(download_folder):
            os.mkdir(download_folder)

        filename = attachment['filename']
        file_path = os.path.join(download_folder, filename)

        try:
            with open(file_path, "wb") as f:
                f.write(attachment['content'])
            print(f"Attachment saved to {file_path}")
            return file_path
        except Exception as e:
            print(f"Error saving attachment: {e}")
            return None

    def process_emails(self):
        """Main function to process incoming emails"""
        try:
            imap = self.connect_imap()
            if not imap:
                return

            imap.select('INBOX')
            print("Connected to inbox")

            _, message_numbers = imap.search(None, 'UNSEEN')

            if not message_numbers[0]:
                print("No new messages found")
                return

            for num in message_numbers[0].split():
                try:
                    _, msg_data = imap.fetch(num, '(RFC822)')
                    email_body = msg_data[0][1]
                    email_message = email.message_from_bytes(email_body)

                    sender = parseaddr(email_message['From'])[1]
                    print(f"\nProcessing email from: {sender}")

                    attachments = self.get_attachments(email_message)

                    if attachments:
                        for attachment in attachments:
                            attachment_path = self.save_attachment(attachment)  # Save attachment

                            if attachment_path:
                                output_pdf = call_pdf_generator(attachment_path)
                                output_excel = call_excel_generator(attachment_path)
                                print(f"output_excel {output_excel}")
                                if output_pdf and output_excel:
                                    self.send_attachments_back(sender, output_pdf, output_excel)
                                    print(f"Attachments sent back to {sender}")

                except Exception as e:
                    print(f"Error processing email: {str(e)}")
                    continue

            imap.close()
            imap.logout()
            print("\nEmail processing completed")

        except Exception as e:
            print(f"Error: {str(e)}")

    def send_attachments_back(self, recipient, output_pdf, output_excel):
        """Send attachments back to the original sender"""
        try:
            msg = MIMEMultipart()
            msg['From'] = self.EMAIL
            msg['To'] = recipient
            msg['Subject'] = "Re: Your Email Attachments"
    
            body = "Here are the processed files based on your email attachment."
            msg.attach(MIMEText(body, 'plain'))
            if output_excel and os.path.exists(output_excel):  
                print(f"Adding Excel attachment: {output_excel}")  
                with open(output_excel, 'rb') as f:
                    part = MIMEBase('application', 'octet-stream')
                    part.set_payload(f.read())
                    encoders.encode_base64(part)
                    part.add_header('Content-Disposition', f"attachment; filename={os.path.basename(output_excel)}")
                    msg.attach(part)
            else:
                print(f"Error: Excel file not found or path is incorrect: {output_excel}")
            if output_pdf and os.path.exists(output_pdf): 
                print(f"Adding PDF attachment: {output_pdf}")  
                with open(output_pdf, 'rb') as f:
                    part = MIMEBase('application', 'octet-stream')
                    part.set_payload(f.read())
                    encoders.encode_base64(part)
                    part.add_header('Content-Disposition', f"attachment; filename={os.path.basename(output_pdf)}")
                    msg.attach(part)
            else:
                print(f"Error: PDF file not found or path is incorrect: {output_pdf}")
            smtp = self.connect_smtp()
            if smtp:
                smtp.send_message(msg)
                smtp.quit()
                print(f"Attachments sent to {recipient}")
            else:
                print("Failed to connect to SMTP server")
    
        except Exception as e:
            print(f"Error sending attachments: {str(e)}")

def call_pdf_generator(input_pdf_path):
    try:
        result = subprocess.run(
            ["python", "pdf_info.py", input_pdf_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
        )
        if result.returncode == 0:
            output_pdf_path = result.stdout.strip()
            print(f"Generated PDF: {output_pdf_path}")
            return output_pdf_path
        else:
            print(f"Error: {result.stderr}")
            return None
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None
def call_excel_generator(input_pdf_path):
    try:
        result = subprocess.run(
            ["python", "excel_info.py", input_pdf_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
        )
        if result.returncode == 0:
            output_excel_path = result.stdout.strip()
            print(f"Generated Excel: {output_excel_path}")
            return output_excel_path
        else:
            print(f"Error: {result.stderr}")
            return None
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

if __name__ == "__main__":
    print("Starting email attachment handler... Press 'q' to quit.")
    handler = GmailHandler()

    try:
        while True:
            handler.process_emails()
            print("Waiting for new emails...")

            # Check for 'q' key press to quit
            if msvcrt.kbhit():
                key = msvcrt.getch().decode('utf-8')
                if key == 'q':
                    print("\nQuitting program...")
                    break

            time.sleep(5) 
    except KeyboardInterrupt:
        print("\nProgram interrupted. Exiting...")


Starting email attachment handler... Press 'q' to quit.
Connected to inbox
No new messages found
Waiting for new emails...
Connected to inbox
No new messages found
Waiting for new emails...
Connected to inbox
No new messages found
Waiting for new emails...
Connected to inbox
No new messages found
Waiting for new emails...
Connected to inbox
No new messages found
Waiting for new emails...
Connected to inbox
No new messages found
Waiting for new emails...
Connected to inbox
No new messages found
Waiting for new emails...
Connected to inbox

Processing email from: 953621104043@ritrjpm.ac.in
Found attachment: input_sheet.pdf
Attachment saved to attachments\input_sheet.pdf
Generated PDF: attachments\new_10.pdf
Generated Excel: attachments/final_2219bdea-cbc3-4503-a1d3-6309bdd95529.xlsx
output_excel attachments/final_2219bdea-cbc3-4503-a1d3-6309bdd95529.xlsx
Adding Excel attachment: attachments/final_2219bdea-cbc3-4503-a1d3-6309bdd95529.xlsx
Adding PDF attachment: attachments\new_10.pdf
Atta